#### Env

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install kafka-python findspark pyspark torch transformers==4.53.0 fuzzywuzzy stanza unidecode python-Levenshtein -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.8/309.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 754.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install pyngrok

In [4]:
import json
import os
from kafka import KafkaProducer
from tqdm import tqdm
from time import sleep, time
import json
import psycopg2
from pyspark.sql import DataFrame

In [5]:
postgres_url = "jdbc:postgresql://ep-mute-lab-a1al1nx2-pooler.ap-southeast-1.aws.neon.tech/socialtrend?sslmode=require"
postgres_properties = {
    "user": "socialtrend_owner",
    "password": "npg_wOv01JkxVquo",
    "driver": "org.postgresql.Driver"
}

PG_CONN_STR = (
    "host=ep-mute-lab-a1al1nx2-pooler.ap-southeast-1.aws.neon.tech "
    "dbname=socialtrend user=socialtrend_owner password=npg_wOv01JkxVquo "
    "sslmode=require channel_binding=require"
)

In [6]:
!wget https://jdbc.postgresql.org/download/postgresql-42.6.0.jar

--2025-07-14 02:25:53--  https://jdbc.postgresql.org/download/postgresql-42.6.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1.0M) [application/java-archive]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1.03M  1.30MB/s    in 0.8s    

2025-07-14 02:25:55 (1.30 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]



In [7]:
!pwd
!wget https://dlcdn.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
!tar -xzf spark-3.5.6-bin-hadoop3.tgz
!pip install findspark
!pip install pyspark kafka-python

/content
--2025-07-14 02:25:55--  https://dlcdn.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400923510 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.6-bin-hadoop3.tgz’

spark-3.5.6-bin-had 100%[===================>] 382.35M  46.0MB/s    in 8.8s    

2025-07-14 02:26:04 (43.6 MB/s) - ‘spark-3.5.6-bin-hadoop3.tgz’ saved [400923510/400923510]



In [8]:
!wget https://downloads.apache.org/kafka/3.7.2/kafka_2.12-3.7.2.tgz
!tar -xzf kafka_2.12-3.7.2.tgz

--2025-07-14 02:26:30--  https://downloads.apache.org/kafka/3.7.2/kafka_2.12-3.7.2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120886836 (115M) [application/x-gzip]
Saving to: ‘kafka_2.12-3.7.2.tgz’

kafka_2.12-3.7.2.tg 100%[===================>] 115.29M  11.5MB/s    in 11s     

2025-07-14 02:26:42 (10.2 MB/s) - ‘kafka_2.12-3.7.2.tgz’ saved [120886836/120886836]



In [9]:
!./kafka_2.12-3.7.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.7.2/config/zookeeper.properties
!./kafka_2.12-3.7.2/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.7.2/config/server.properties

In [10]:
import time
print("Waiting 10 seconds for Kafka and Zookeeper to be ready...")
time.sleep(10)

Waiting 10 seconds for Kafka and Zookeeper to be ready...


In [11]:
!ps -ef | grep kafka

root        2847       1 17 02:26 ?        00:00:01 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-3.7.2/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-3.7.2/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-3.7.2/bin/../config/log4j.properties -cp /content/kafka_2.12-3.7.2/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-3.7.2/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-3.7.2/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-3.7.2/bin/../libs/audience-annotations-0.12.0.jar:/content/kafka_2.12-3.7.2/bin/../libs/caffeine-2.9.3.jar:/content/kafka_2.12-3.7.2/bin/../libs/checker-qual-3.19.0.jar:/content/kafka_2.12

In [12]:
!./kafka_2.12-3.7.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 \
    --replication-factor 1 --partitions 1 --topic news-stream

Created topic news-stream.


In [13]:
!./kafka_2.12-3.7.2/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic news-stream

Topic: news-stream	TopicId: AuOVcEghSdCpF4iAj-4EFA	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: news-stream	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


#### Consumer

In [14]:
# !pip install --upgrade transformers

In [15]:
import os, json, torch, findspark
from pyspark.sql import SparkSession
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import from_json, col, pandas_udf, udf, struct
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from threading import Thread
import pickle

In [16]:
os.environ["SPARK_HOME"] = "/content/spark-3.5.6-bin-hadoop3"
findspark.init()

In [17]:
spark_jars_packages = (
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.6,"
    "org.postgresql:postgresql:42.6.0"
)

In [18]:
    # .config("spark.driver.memory", "3g") \
    # .config("spark.executor.memory", "3g") \
    # .config("spark.sql.shuffle.partitions", "2") \
    # .config("spark.default.parallelism", "2") \
    # .config("spark.sql.execution.arrow.pyspark.enabled", "true") \

In [19]:
spark = SparkSession.builder \
    .appName("KafkaSparkABSA_NER") \
    .config("spark.jars.packages", spark_jars_packages) \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "5g") \
    .getOrCreate()

In [20]:
schema = StructType([StructField("text", StringType(), True)])

df_raw = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "localhost:9092")
         .option("subscribe", "news-stream")
         .option("startingOffsets", "earliest")
        #  .option("maxOffsetsPerTrigger", "20")
         .load()
)

In [21]:
def build_text(item):
    parts = [item.get("post_content", "")]
    if "comment" in item:
        parts += item["comment"].get("parent_comment_texts", [])
        ct = item["comment"].get("comment_text", "")
        if ct:
            parts.append(ct)
    parts += [d.get("image_description", "") for d in item.get("image_descriptions", [])]
    return "\n".join([p for p in parts if p])

In [22]:
@udf(StringType())
def extract_all_fields(json_str):
    try:
        item = json.loads(json_str)
        text_all = build_text(item)
        text_content = item.get("post_content", "")
        creation_time = str(item.get("creation_time", ""))
        total_reactions = int(item.get("total_reactions", 0))
        share_count = int(item.get("share_count", 0))
        return json.dumps({
            "text_all": text_all,
            "text_content": text_content,
            "creation_time": creation_time,
            "total_reactions": total_reactions,
            "share_count": share_count
        })
    except:
        return json.dumps({
            "text_all": "",
            "text_content": "",
            "creation_time": "",
            "total_reactions": 0,
            "share_count": 0
        })

In [23]:
schema = StructType([
    StructField("text_all", StringType(), True),
    StructField("text_content", StringType(), True),
    StructField("creation_time", StringType(), True),
    StructField("total_reactions", StringType(), True),
    StructField("share_count", StringType(), True)
])

In [24]:
df_extracted = (
    df_raw
      .selectExpr("CAST(value AS STRING) as json_str")
      .withColumn("json_cleaned", extract_all_fields(col("json_str")))
)

In [25]:
df = (
    df_extracted
      .withColumn("data", from_json(col("json_cleaned"), schema))
      .select("data.*")
)

In [ ]:
def start_producer():
    # sleep(20)
    print("Starting producer...")
    producer = KafkaProducer(
        bootstrap_servers='localhost:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    data_files = [
    '/content/drive/MyDrive/ds200/final_project/data_test/K14vn.json',
    '/content/drive/MyDrive/ds200/final_project/data_test/vietnamnet.json',
    '/content/drive/MyDrive/ds200/final_project/data_test/Theanh28.json'
    ]
    data = []
    for file in data_files:
        with open(file) as f:
            data += json.load(f)

    print(f"Sending {len(data)} items to Kafka topic 'news-stream'...")
    # data = data[:36]
    for item in data:
        if stop_flag.is_set():
          print("Producer stopped early")
          break
        producer.send("news-stream", item)
        # print(f"→ Sent: {item.get('post_content', '')[:50]}...")
        sleep(10)

    producer.flush()
    print("Producer done sending.")

In [27]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [28]:
from fuzzywuzzy import fuzz

In [29]:
df.printSchema()

root
 |-- text_all: string (nullable = true)
 |-- text_content: string (nullable = true)
 |-- creation_time: string (nullable = true)
 |-- total_reactions: string (nullable = true)
 |-- share_count: string (nullable = true)



hàm make_predict_fn() này dùng strategy tính embedding cho câu trước rồi mới vào phần mô hình nằm trong class

In [30]:
def make_predict_fn():
    import torch
    import torch.nn as nn
    import json, joblib, stanza, re
    from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification, pipeline
    from unidecode import unidecode
    from fuzzywuzzy import fuzz

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # === Load NER ===
    tokenizer_ner = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
    model_ner = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
    ner_pipeline = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, aggregation_strategy="simple")

    # === Load Stanza ===
    stanza.download('vi')
    nlp_stanza = stanza.Pipeline('vi', processors='tokenize,pos,lemma,depparse')

    # === Load alias mapping ===
    alias_path = "/content/drive/MyDrive/ds200/final_project/celebrity_aliases.json"
    with open(alias_path, encoding="utf-8") as f:
        alias_dict = json.load(f)

    # === Load ABSA ===
    MODEL_DIR = "/content/drive/MyDrive/ds200/final_project/model/saved_model"
    tokenizer_embed = AutoTokenizer.from_pretrained(MODEL_DIR)
    model_token = AutoModel.from_pretrained(MODEL_DIR).to(device).eval()
    aspect_encoder = joblib.load("/content/drive/MyDrive/ds200/final_project/model/aspect_encoder.pkl")
    sentiment_encoder = joblib.load("/content/drive/MyDrive/ds200/final_project/model/sentiment_encoder.pkl")

    with open("/content/drive/MyDrive/ds200/final_project/model/classifier_config.json") as f:
        cfg = json.load(f)

    class MultiTaskClassifier(nn.Module):
        def __init__(self, input_dim, lstm_hidden_dim, lstm_layers, hidden_dims,
                     num_aspects, num_sentiments, task_specific_dims=None):
            super().__init__()
            self.bilstm = nn.LSTM(input_dim, lstm_hidden_dim, lstm_layers,
                                  batch_first=True, bidirectional=True)
            layers, prev = [], lstm_hidden_dim * 2
            for h in hidden_dims:
                layers += [nn.Linear(prev, h), nn.ReLU(), nn.Dropout(0.3), nn.BatchNorm1d(h)]
                prev = h
            self.shared = nn.Sequential(*layers)
            if task_specific_dims is None:
                task_specific_dims = [prev // 2, prev // 4]
            self.aspect_1_layers = self._head(prev, task_specific_dims, num_aspects)
            self.aspect_2_layers = self._head(prev, task_specific_dims, num_aspects)
            self.sentiment_1_layers = self._head(prev, task_specific_dims, num_sentiments)
            self.sentiment_2_layers = self._head(prev, task_specific_dims, num_sentiments)

        def _head(self, inp, hs, out_dim):
            layers = []
            for h in hs:
                layers += [nn.Linear(inp, h), nn.ReLU(), nn.Dropout(0.2)]
                inp = h
            layers.append(nn.Linear(inp, out_dim))
            return nn.Sequential(*layers)

        def forward(self, x):
            out, _ = self.bilstm(x)
            pooled = out.mean(1)
            shared = self.shared(pooled)
            return (self.aspect_1_layers(shared), self.aspect_2_layers(shared),
                    self.sentiment_1_layers(shared), self.sentiment_2_layers(shared))

    classifier = MultiTaskClassifier(**cfg).to(device)
    classifier.load_state_dict(torch.load(
        "/content/drive/MyDrive/ds200/final_project/model/classifier.pt", map_location=device
    ))
    classifier.eval()

    def load_alias_mapping(alias_path):
        with open(alias_path, "r", encoding="utf-8") as f:
            alias_dict = json.load(f)
        alias_to_name = {}
        for name, aliases in alias_dict.items():
            alias_to_name[unidecode(name).lower()] = name
            for alias in aliases:
                alias_to_name[unidecode(alias).lower()] = name
        return alias_to_name

    def replace_alias_with_canonical_in_subjects(text, alias_to_name, subjects):
        for subj in subjects:
            subj_norm = unidecode(subj).lower()
            for canonical_name, aliases in alias_to_name.items():
                aliases_norm = [unidecode(alias).lower() for alias in aliases]
                if subj_norm in aliases_norm:
                    text = text.replace(subj, canonical_name)
                    break
        return text

    def extract_subjects(doc):
        subjects = []
        for sent in doc.sentences:
            for word in sent.words:
                if word.deprel in ["nsubj", "nsubj:pass"]:
                    phrase = [word.text]
                    for w in sent.words:
                        if w.head == word.id and w.deprel in ["compound", "flat", "nmod"]:
                            phrase.append(w.text)
                    phrase_sorted = sorted(set(phrase), key=lambda x: doc.text.lower().find(x.lower()))
                    subject_phrase = " ".join(phrase_sorted)
                    subjects.append(subject_phrase)
        return list(set(subjects))

    def capitalize_subjects(text, subjects):
        for subj in subjects:
            if subj.strip():
                text = re.sub(r'\b' + re.escape(subj) + r'\b', subj.title(), text, flags=re.IGNORECASE)
        return text

    def resolve_entity_to_celebrity_fuzzy(entity_word, alias_dict, threshold=90):
        word_clean = unidecode(entity_word)
        best_score = 0
        best_match = None
        for canonical, aliases in alias_dict.items():
            for alias in aliases:
                alias_clean = unidecode(alias)
                score = fuzz.ratio(word_clean, alias_clean)
                if score > best_score and score >= threshold:
                    best_score = score
                    best_match = canonical
        return best_match

    def preprocess_and_ner(text):
        alias_to_name = load_alias_mapping(alias_path)

        # Phân tích ban đầu để tìm chủ ngữ
        doc = nlp_stanza(text)
        subjects = extract_subjects(doc)

        # Chỉ thay alias trong cụm chủ ngữ
        text = replace_alias_with_canonical_in_subjects(text, alias_to_name, subjects)

        # Phân tích lại sau khi thay thế alias
        doc = nlp_stanza(text)
        subjects = extract_subjects(doc)

        # Viết hoa các chủ ngữ
        text_modified = capitalize_subjects(text, subjects)

        # Dự đoán NER
        ner_results = ner_pipeline(text_modified)
        print(ner_results)
        person_entities = [ent for ent in ner_results if ent["entity_group"] == "PERSON"]
        print(person_entities)
        resolved_entities = []
        seen = set()

        for ent in person_entities:
            word = ent["word"]
            canonical = resolve_entity_to_celebrity_fuzzy(word, alias_dict)
            name = canonical if canonical else word
            if name not in seen:
                resolved_entities.append(name)
                seen.add(name)

        return resolved_entities

    def predict(batch):
        import numpy as np
        results = []
        rows = batch.tolist() if isinstance(batch, np.ndarray) else list(batch)

        for row in rows:
            text_all = row[0] or ""
            text_content = row[1] or ""

            # Sử dụng preprocess_and_ner trực tiếp
            ner_content = preprocess_and_ner(text_content)
            ner_all = preprocess_and_ner(text_all)

            enc = tokenizer_embed(
                [text_all],
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=256
            ).to(device)
            with torch.no_grad():
                emb = model_token(**enc).last_hidden_state
                a1, a2, s1, s2 = classifier(emb)
            absa = {
                "aspect1": aspect_encoder.classes_[int(a1[0].argmax())],
                "aspect2": aspect_encoder.classes_[int(a2[0].argmax())],
                "sent1": sentiment_encoder.classes_[int(s1[0].argmax())],
                "sent2": sentiment_encoder.classes_[int(s2[0].argmax())],
            }

            results.append({
                "ner_content": json.dumps(ner_content, ensure_ascii=False),
                "ner_all": json.dumps(ner_all, ensure_ascii=False),
                "aspect1": absa["aspect1"],
                "aspect2": absa["aspect2"],
                "sent1": absa["sent1"],
                "sent2": absa["sent2"],
            })

        return results

    return predict

dùng make_predict_fn() ở dưới cho 2 strategy cho hết cả embedding và cả mô hình vào trong 1 class

In [31]:
# def make_predict_fn():
#     import torch
#     import torch.nn as nn
#     import json, joblib, stanza, re
#     from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification, pipeline
#     from unidecode import unidecode
#     from fuzzywuzzy import fuzz

#     device = "cuda" if torch.cuda.is_available() else "cpu"

#     # === Load NER model ===
#     tokenizer_ner = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
#     model_ner = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base").to(device)
#     ner_pipeline = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, aggregation_strategy="simple")

#     # === Load Stanza for dependency parsing ===
#     stanza.download('vi')
#     nlp_stanza = stanza.Pipeline('vi', processors='tokenize,pos,lemma,depparse')

#     # === Load alias mapping ===
#     alias_path = "/content/drive/MyDrive/ds200/final_project/celebrity_aliases.json"
#     with open(alias_path, encoding="utf-8") as f:
#         alias_dict = json.load(f)

#     def load_alias_mapping(d):
#         inv = {}
#         for name, aliases in d.items():
#             inv[name.lower()] = name
#             for a in aliases:
#                 inv[a.lower()] = name
#         return inv

#     alias_to_name = load_alias_mapping(alias_dict)

#     # === Load checkpoint & model config ===
#     ckpt = torch.load("/content/drive/MyDrive/ds200/final_project/model/saved_model/model.pt", map_location=device)
#     config_keys = {k: ckpt[k] for k in ckpt if k != 'model_state_dict'}

#     MODEL_DIR = "/content/drive/MyDrive/ds200/final_project/model/saved_model"

#     # === Define classifier ===
#     class MultiTaskClassifierWithEmbedding(nn.Module):
#         def __init__(self, model_name, lstm_hidden_dim, lstm_layers,
#                      hidden_dims, num_aspects, num_sentiments,
#                      task_specific_dims=None, device='cuda'):
#             super().__init__()
#             self.device = device
#             self.tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
#             self.embedding_model = AutoModel.from_pretrained(MODEL_DIR).to(device).eval()
#             for p in self.embedding_model.parameters():
#                 p.requires_grad = False

#             input_dim = self.embedding_model.config.hidden_size
#             self.bilstm = nn.LSTM(input_dim, lstm_hidden_dim, lstm_layers,
#                                   batch_first=True, bidirectional=True)
#             prev = lstm_hidden_dim * 2
#             layers = []
#             for h in hidden_dims:
#                 layers += [nn.Linear(prev, h), nn.ReLU(), nn.Dropout(0.3), nn.BatchNorm1d(h)]
#                 prev = h
#             self.shared = nn.Sequential(*layers)
#             if task_specific_dims is None:
#                 task_specific_dims = [prev // 2, prev // 4]
#             self.aspect_1_layers = self._create_head(prev, task_specific_dims, num_aspects)
#             self.aspect_2_layers = self._create_head(prev, task_specific_dims, num_aspects)
#             self.sentiment_1_layers = self._create_head(prev, task_specific_dims, num_sentiments)
#             self.sentiment_2_layers = self._create_head(prev, task_specific_dims, num_sentiments)

#         def _create_head(self, inp, hid, out):
#             layers = []
#             prev = inp
#             for h in hid:
#                 layers += [nn.Linear(prev, h), nn.ReLU(), nn.Dropout(0.2)]
#                 prev = h
#             layers.append(nn.Linear(prev, out))
#             return nn.Sequential(*layers)

#         def forward(self, texts):
#             with torch.no_grad():
#                 enc = self.tokenizer(texts, return_tensors='pt',
#                                      truncation=True, padding='max_length',
#                                      max_length=512).to(self.device)
#                 emb = self.embedding_model(**enc).last_hidden_state
#             lstm_out, _ = self.bilstm(emb)
#             pooled = lstm_out.mean(1)
#             shared = self.shared(pooled)
#             return (self.aspect_1_layers(shared),
#                     self.aspect_2_layers(shared),
#                     self.sentiment_1_layers(shared),
#                     self.sentiment_2_layers(shared))

#     # === Instantiate & load model ===
#     classifier = MultiTaskClassifierWithEmbedding(**config_keys, device=device).to(device)
#     # Bỏ qua embedding_model.* khi load
#     filtered_state_dict = {
#         k: v for k, v in ckpt['model_state_dict'].items()
#         if not k.startswith('embedding_model.')
#     }
#     classifier.load_state_dict(filtered_state_dict, strict=False)
#     classifier.eval()

#     # === Load label encoders ===
#     aspect_encoder = joblib.load("/content/drive/MyDrive/ds200/final_project/model/aspect_encoder.pkl")
#     sentiment_encoder = joblib.load("/content/drive/MyDrive/ds200/final_project/model/sentiment_encoder.pkl")

#     # === NER preprocessing helpers ===
#     def extract_subjects(doc):
#         subs = []
#         for sent in doc.sentences:
#             for w in sent.words:
#                 if w.deprel in ["nsubj", "nsubj:pass"]:
#                     phrase = [w.text] + [c.text for c in sent.words if c.head == w.id and c.deprel in ["compound", "flat", "nmod"]]
#                     subs.append(" ".join(sorted(set(phrase), key=lambda x: doc.text.find(x))))
#         return list(set(subs))

#     def capitalize_subjects(text, subjects):
#         for s in subjects:
#             if s.strip():
#                 text = re.sub(r'\b' + re.escape(s) + r'\b', s.title(), text, flags=re.IGNORECASE)
#         return text

#     def resolve_entity(entity):
#         best, name = 0, None
#         clean = unidecode(entity)
#         for canon, aliases in alias_dict.items():
#             for a in aliases:
#                 score = fuzz.ratio(clean, unidecode(a))
#                 if score > best and score >= 90:
#                     best, name = score, canon
#         return name or entity

#     def preprocess_and_ner(text):
#         doc = nlp_stanza(text)
#         subs = extract_subjects(doc)
#         for subj in subs:
#             norm = unidecode(subj).lower()
#             if norm in alias_to_name:
#                 text = text.replace(subj, alias_to_name[norm])
#         doc = nlp_stanza(text)
#         text = capitalize_subjects(text, extract_subjects(doc))
#         ents = ner_pipeline(text)
#         persons = [e["word"] for e in ents if e["entity_group"] == "PERSON"]
#         seen, resolved = set(), []
#         for p in persons:
#             r = resolve_entity(p)
#             if r not in seen:
#                 resolved.append(r)
#                 seen.add(r)
#         return resolved

#     # === Final predict ===
#     def predict(batch):
#         import numpy as np
#         results = []
#         rows = batch.tolist() if isinstance(batch, np.ndarray) else list(batch)

#         for row in rows:
#             text_all = row[0] or ""
#             text_content = row[1] or ""

#             # NER cho từng phần
#             ner_content = preprocess_and_ner(text_content)
#             ner_all = preprocess_and_ner(text_all)

#             # Dự đoán ABSA dùng text_all
#             a1_logits, a2_logits, s1_logits, s2_logits = classifier([text_all])
#             absa = {
#                 "aspect1": aspect_encoder.classes_[int(a1_logits.argmax())],
#                 "aspect2": aspect_encoder.classes_[int(a2_logits.argmax())],
#                 "sent1": sentiment_encoder.classes_[int(s1_logits.argmax())],
#                 "sent2": sentiment_encoder.classes_[int(s2_logits.argmax())]
#             }

#             # Append kết quả
#             results.append({
#                 "ner_content": json.dumps(ner_content, ensure_ascii=False),
#                 "ner_all": json.dumps(ner_all, ensure_ascii=False),
#                 "aspect1": absa["aspect1"],
#                 "aspect2": absa["aspect2"],
#                 "sent1": absa["sent1"],
#                 "sent2": absa["sent2"],
#             })

#         return results

#     return predict


In [32]:
def write_full_batch(batch_df: DataFrame, batch_id: int):
    import psycopg2
    import json
    global seen_creation_times

    def normalize(val):
      if val is None:
          return None
      val_str = str(val).strip().lower()
      if val_str in ("", "null", "none"):
          return None
      return val

    conn = psycopg2.connect(PG_CONN_STR)
    cur  = conn.cursor()

    for row in batch_df.toLocalIterator():
        creation_time = getattr(row, "creation_time", None)
        ner_all       = json.loads(getattr(row, "ner_all", "[]"))
        aspect1       = normalize(getattr(row, "aspect1", None))
        aspect2       = normalize(getattr(row, "aspect2", None))
        sent1         = normalize(getattr(row, "sent1", None).lower())
        sent2         = normalize(getattr(row, "sent2", None).lower())

        try:
            total_reacts = int(getattr(row, "total_reactions", 0))
            share_counts = int(getattr(row, "share_count", 0))
        except:
            total_reacts = 0
            share_counts = 0

        total_reacts += share_counts

        absa_pairs = []
        if aspect1 and sent1: absa_pairs.append((aspect1, sent1))
        if aspect2 and sent2: absa_pairs.append((aspect2, sent2))

        for celeb_name in ner_all:
            cur.execute("""
                SELECT id FROM public.celebrities WHERE name = %s
                UNION
                SELECT c.id FROM public.celebrity_aliases a
                JOIN public.celebrities c ON a.celebrity_id = c.id
                WHERE a.alias = %s
                LIMIT 1
            """, (celeb_name, celeb_name))
            fetch = cur.fetchone()

            if fetch:
                celeb_id = fetch[0]
            else:
                cur.execute("""
                    INSERT INTO public.celebrities (name, image_url, created_at, is_celebrity)
                    VALUES (%s, %s, now(), FALSE)
                    RETURNING id
                """, (celeb_name, None))
                celeb_id = cur.fetchone()[0]

            if creation_time and creation_time not in seen_creation_times:
                seen_creation_times.add(creation_time)
                cur.execute("""
                    INSERT INTO public.reactions (celebrity_id, total_reactions, created_at)
                    VALUES (%s, %s, now())
                """, (celeb_id, total_reacts))

            for aspect, sentiment in absa_pairs:
                inc_pos = 1 if sentiment == "positive" else 0
                inc_neg = 1 if sentiment == "negative" else 0
                inc_neu = 1 if sentiment == "neutral"  else 0
                # print(aspect)
                cur.execute("""
                    INSERT INTO public.interactions
                      (celebrity_id, positive_count, negative_count,
                       neutral_count, field, interaction_date, created_at)
                    VALUES (%s, %s, %s, %s, %s, CURRENT_DATE, now())
                """, (celeb_id, inc_pos, inc_neg, inc_neu, aspect))

    conn.commit()
    cur.close()
    conn.close()


In [33]:
query = (
    df.writeStream
      .outputMode("append")
      .format("memory")
      .queryName("debug_text")
      .trigger(processingTime="2 seconds")
      .start()
)

In [34]:
seen_creation_times = set()

In [35]:
absa_ner_schema = StructType([
    StructField("ner_content", StringType(), True),
    StructField("ner_all", StringType(), True),
    StructField("aspect1", StringType(), True),
    StructField("aspect2", StringType(), True),
    StructField("sent1", StringType(), True),
    StructField("sent2", StringType(), True),
])

predict_udf = predict_batch_udf(
    make_predict_fn,
    return_type=absa_ner_schema,
    batch_size=4
)

In [36]:
df_pred = df.withColumn("pred_struct", predict_udf(struct("text_all", "text_content"))) \
            .select("*", "pred_struct.*") \
            .drop("pred_struct")

In [37]:
debug_pred_query = (
    df_pred.writeStream
        .outputMode("append")
        .format("memory")
        .queryName("debug_pred")
        .trigger(processingTime="2 seconds")
        .start()
)

In [38]:
streaming_predict = (
    df_pred.writeStream
        .foreachBatch(write_full_batch)
        .outputMode("append")
        .trigger(processingTime="2 seconds")
        .start()
)

In [39]:
import time
from IPython.display import clear_output, display

In [40]:
import threading
stop_flag = threading.Event()

In [41]:
producer_thread = Thread(target=start_producer)

In [42]:
print(f"Thread name: {producer_thread.name}")
print(f"Thread is alive: {producer_thread.is_alive()}")
print(f"Thread ID: {producer_thread.ident}")


Thread name: Thread-12 (start_producer)
Thread is alive: False
Thread ID: None


In [43]:
producer_thread.start()

Starting producer...


In [44]:
time.sleep(120)

Sending 177 items to Kafka topic 'news-stream'...


In [45]:
print(f"Thread name: {producer_thread.name}")
print(f"Thread is alive: {producer_thread.is_alive()}")
print(f"Thread ID: {producer_thread.ident}")


Thread name: Thread-12 (start_producer)
Thread is alive: True
Thread ID: 140658204755520


In [46]:
pd.set_option("display.max_colwidth", None)
for i in range(8):
    clear_output(wait=True)
    df_live = spark.sql("SELECT * FROM debug_text")
    print(f"Live view at {(i)*10} seconds:")
    display(df_live.toPandas())
    time.sleep(10)

Live view at 70 seconds:


,text_all,text_content,creation_time,total_reactions,share_count
0,"Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nTheo các dữ liệu được Cổng thông tin Quốc gia về Đăng ký doanh nghiệp công bố đối với Công ty CP Sản xuất và Thương mại EHERBAL cho thấy, vợ chồng diễn viên Hữu Công (ông Nguyễn Hữu Công và bà Phan Thủy Tiên) hiện không sở hữu cổ phần hay tham gia đại diện pháp luật tại doanh nghiệp này 👉 https://s.biz.vn/Ar8KglZ",Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99
1,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nMỗi ngày thêm 1 thành viên mới 🥲,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99


In [47]:
pd.set_option("display.max_colwidth", None)
for i in range(8):
    clear_output(wait=True)
    df_show = spark.sql("SELECT * FROM debug_pred")
    print(f"{(i)*10} giây")
    display(df_show.toPandas())
    time.sleep(10)

70 giây


,text_all,text_content,creation_time,total_reactions,share_count,ner_content,ner_all,aspect1,aspect2,sent1,sent2
0,"Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nTheo các dữ liệu được Cổng thông tin Quốc gia về Đăng ký doanh nghiệp công bố đối với Công ty CP Sản xuất và Thương mại EHERBAL cho thấy, vợ chồng diễn viên Hữu Công (ông Nguyễn Hữu Công và bà Phan Thủy Tiên) hiện không sở hữu cổ phần hay tham gia đại diện pháp luật tại doanh nghiệp này 👉 https://s.biz.vn/Ar8KglZ",Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công"", ""Viên"", ""Phan Thủy Tiên""]",Law,Other,negative,positive
1,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nMỗi ngày thêm 1 thành viên mới 🥲,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công""]",Health,Food,positive,negative
2,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nMỗi ngày thêm 1 thành viên mới 🥲\nYến Lâm năm nay tiktoker thay thế nghề kế toán rồi,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công"", ""Yến Lâm"", ""Tiktoker""]",Food,Law,neutral,neutral
3,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nChỉ nể ng mua hàng,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công"", ""Ng""]",Food,Law,positive,neutral
4,"Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nChỉ nể ng mua hàng\nĐào Trung thiệt, nể mấy Má gặp ai cũng tin, gặp ai bán cũng mua, niềm tin thiệt là …..",Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công""]",Sport,Food,positive,neutral
5,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nChỉ nể ng mua hàng\nĐào Trung chưa biết mặt hàng này hjj,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công"", ""Ng""]",Food,Law,negative,negative
6,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nChỉ nể ng mua hàng\nĐào Trung động vào nỗi đau của tôi cứ nghĩ tốt cho sk lại hại bản thân.làm giàu cho chúng nó,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công"", ""Ng""]",Health,Law,positive,neutral
7,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴\nChỉ nể ng mua hàng\nTrinh Phạm Văn tin ng quá và niềm tin đặt sai chỗ,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm rộ trên mạng xã hội 🥴,1750898193,6292,99,"[""Nguyễn Hữu Công""]","[""Nguyễn Hữu Công"", ""Trinh Phạm Văn""]",Food,Law,neutral,neutral
8,Thông tin về doanh nghiệp chỉ có 6 lao động đứng sau loạt sản phẩm Detox được vợ chồng Hữu Công quảng bá rầm

In [48]:
spark.sql("SELECT COUNT(*) FROM debug_pred").show()

+--------+
|count(1)|
+--------+
|      28|
+--------+



In [49]:
query.stop()

In [50]:
debug_pred_query.stop()

In [51]:
streaming_predict.stop()

In [52]:
stop_flag.set()
producer_thread.join()

Producer stopped early
Producer done sending.


In [53]:
import time

In [54]:
from pyspark.sql import Row
import time

sample_records = [
    Row(
        text_all="Ca sĩ Mỹ Tâm vừa ra mắt MV mới. Cô ấy được khen ngợi rất nhiều. Mỹ Tâm quá tuyệt vời!",
        text_content="Ca sĩ Mỹ Tâm vừa ra mắt MV mới.",
        creation_time=str(int(time.time())),
        total_reactions="1200"
    ),
    Row(
        text_all="Sơn Tùng M-TP bị chỉ trích vì lời bài hát. Không nên chỉ trích Sơn Tùng như vậy.",
        text_content="Sơn Tùng M-TP bị chỉ trích vì lời bài hát.",
        creation_time=str(int(time.time())),
        total_reactions="900"
    )
]

test_df = spark.createDataFrame(sample_records)


In [55]:
# Khởi tạo hàm predict
predict = make_predict_fn()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: vi (Vietnamese) ...
INFO:stanza:File exists: /root/stanza_resources/vi/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: vi (Vietnamese):
| Processor | Package    |
--------------------------
| tokenize  | vtb        |
| pos       | vtb_charlm |
| lemma     | identity   |
| depparse  | vtb_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!
Some weights of XLMRobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/ds200/final_project/model/saved_model and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.atte

In [56]:
batch = test_df.select("text_all", "text_content").rdd.map(lambda r: (r["text_all"], r["text_content"])).collect()

In [57]:
# Dự đoán
outputs = predict(batch)

# In kết quả
import pandas as pd
pd.DataFrame(outputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'PERSON', 'score': np.float32(0.9992484), 'word': 'Mỹ Tâm', 'start': 6, 'end': 12}]
[{'entity_group': 'PERSON', 'score': np.float32(0.9992484), 'word': 'Mỹ Tâm', 'start': 6, 'end': 12}]
[{'entity_group': 'PERSON', 'score': np.float32(0.9995295), 'word': 'Mỹ Tâm', 'start': 6, 'end': 12}, {'entity_group': 'PERSON', 'score': np.float32(0.99947554), 'word': 'Mỹ Tâm', 'start': 64, 'end': 70}]
[{'entity_group': 'PERSON', 'score': np.float32(0.9995295), 'word': 'Mỹ Tâm', 'start': 6, 'end': 12}, {'entity_group': 'PERSON', 'score': np.float32(0.99947554), 'word': 'Mỹ Tâm', 'start': 64, 'end': 70}]


XLMRobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


[{'entity_group': 'PERSON', 'score': np.float32(0.99929726), 'word': 'Sơn Tùng M - Tp', 'start': 0, 'end': 13}]
[{'entity_group': 'PERSON', 'score': np.float32(0.99929726), 'word': 'Sơn Tùng M - Tp', 'start': 0, 'end': 13}]
[{'entity_group': 'PERSON', 'score': np.float32(0.9993553), 'word': 'Sơn Tùng M - Tp', 'start': 0, 'end': 13}, {'entity_group': 'PERSON', 'score': np.float32(0.999515), 'word': 'Sơn Tùng', 'start': 63, 'end': 71}]
[{'entity_group': 'PERSON', 'score': np.float32(0.9993553), 'word': 'Sơn Tùng M - Tp', 'start': 0, 'end': 13}, {'entity_group': 'PERSON', 'score': np.float32(0.999515), 'word': 'Sơn Tùng', 'start': 63, 'end': 71}]


,ner_content,ner_all,aspect1,aspect2,sent1,sent2
0,"[""Mỹ Tâm""]","[""Mỹ Tâm""]",Fashion,Fashion,negative,neutral
1,"[""Sơn Tùng M - Tp""]","[""Sơn Tùng M - Tp"", ""Sơn Tùng""]",Health,Fashion,negative,neutral


In [58]:
batch = test_df.select("text_all").rdd.map(lambda r: r["text_all"]).collect()
outputs = predict(batch)
import pandas as pd
pd.DataFrame(outputs)

[]
[]
[]
[]
[]
[]
[{'entity_group': 'PERSON', 'score': np.float32(0.84630966), 'word': 'S', 'start': 0, 'end': 1}]
[{'entity_group': 'PERSON', 'score': np.float32(0.84630966), 'word': 'S', 'start': 0, 'end': 1}]


,ner_content,ner_all,aspect1,aspect2,sent1,sent2
0,[],[],Fashion,Fashion,negative,negative
1,[],"[""S""]",Fashion,Other,negative,negative


In [59]:
sample_data = [
    {
        "creation_time": str(int(time.time())),
        "ner_all": '["Sơn Tùng M-TP", "Hà Hồ"]',
        "aspect1": "Food",
        "aspect2": "Other",
        "sent1": "positive",
        "sent2": "neutral",
        "total_reactions": 100,
        "share_count": 20,
    },
    {
        "creation_time": str(int(time.time())-10),
        "ner_all": '["Trấn Thành"]',
        "aspect1": "Art",
        "aspect2": "",
        "sent1": "positive",
        "sent2": "",
        "total_reactions": 200,
        "share_count": 50,
    },
]


In [60]:
from pyspark.sql.types import *
from pyspark.sql import Row

rows = [Row(**item) for item in sample_data]
df = spark.createDataFrame(rows)

In [61]:
seen_creation_times = set()

In [62]:
write_full_batch(df, batch_id=0)

In [63]:
# import psycopg2

# conn = psycopg2.connect(PG_CONN_STR)
# cur  = conn.cursor()

# # 1) Liệt kê cột và kiểu
# cur.execute("""
#   SELECT column_name, data_type
#     FROM information_schema.columns
#    WHERE table_schema = 'public'
#      AND table_name   = 'interactions';
# """)
# for col, dtype in cur.fetchall():
#     print(f"{col:20s} | {dtype}")

# # 2) Liệt kê các constraint unique / pk
# cur.execute("""
#   SELECT
#     tc.constraint_name,
#     tc.constraint_type,
#     kcu.column_name
#   FROM information_schema.table_constraints AS tc
#   JOIN information_schema.key_column_usage AS kcu
#     ON tc.constraint_name = kcu.constraint_name
#    AND tc.table_schema   = kcu.table_schema
#   WHERE tc.table_schema = 'public'
#     AND tc.table_name   = 'interactions'
#     AND tc.constraint_type IN ('PRIMARY KEY','UNIQUE');
# """)
# print("\nConstraints:")
# for name, ctype, col in cur.fetchall():
#     print(f"{ctype:12s} | {name:30s} | {col}")

# cur.close()
# conn.close()


In [64]:
# import psycopg2

# # Chuỗi kết nối psycopg2
# PG_CONN_STR = (
#     "host=ep-mute-lab-a1al1nx2-pooler.ap-southeast-1.aws.neon.tech "
#     "dbname=socialtrend user=socialtrend_owner password=npg_wOv01JkxVquo "
#     "sslmode=require channel_binding=require"
# )

# def print_table_info(table_name):
#     conn = psycopg2.connect(PG_CONN_STR)
#     cur  = conn.cursor()

#     print(f"\n=== TABLE: {table_name} ===")

#     # 1) In schema: cột + kiểu
#     cur.execute(f"""
#       SELECT column_name, data_type
#         FROM information_schema.columns
#        WHERE table_schema = 'public'
#          AND table_name   = %s
#        ORDER BY ordinal_position;
#     """, (table_name,))
#     print("Columns:")
#     for col, dtype in cur.fetchall():
#         print(f"  {col:20s} | {dtype}")

#     # 2) In constraints: PK & UNIQUE
#     cur.execute(f"""
#       SELECT
#         tc.constraint_name,
#         tc.constraint_type,
#         string_agg(kcu.column_name, ', ') AS columns
#       FROM information_schema.table_constraints AS tc
#       JOIN information_schema.key_column_usage AS kcu
#         ON tc.constraint_name = kcu.constraint_name
#        AND tc.table_schema   = kcu.table_schema
#       WHERE tc.table_schema = 'public'
#         AND tc.table_name   = %s
#         AND tc.constraint_type IN ('PRIMARY KEY','UNIQUE')
#       GROUP BY tc.constraint_name, tc.constraint_type
#       ORDER BY tc.constraint_type, tc.constraint_name;
#     """, (table_name,))

#     constraints = cur.fetchall()
#     if constraints:
#         print("\nConstraints:")
#         for name, ctype, cols in constraints:
#             print(f"  {ctype:12s} | {name:30s} | {cols}")
#     else:
#         print("\nNo PRIMARY KEY or UNIQUE constraints found.")

#     cur.close()
#     conn.close()

# # In cho hai bảng
# print_table_info("celebrities")
# print_table_info("celebrity_aliases")
# print_table_info("reactions")
